In [ ]:
!pip install -qqq pandas pycaret linear-tree shap sdv explainerdashboard imbalanced-learn

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2022.12.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.1.4 which is incompatible.
xarray-einstats 0.4.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
xarray-einstats 0.4.0 requires scipy>=1.6, but you have scipy 1.5.4 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
tensorflow 2.9.2 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jaxlib 0.3.25+cuda11.cudnn805 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
jax 0.3.25 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.9 which is incompatible.
cmdsta

In [ ]:
!pip show pycaret

Name: pycaret
Version: 2.3.10
Summary: PyCaret - An open source, low-code machine learning library in Python.
Home-page: https://github.com/pycaret/pycaret
Author: Moez Ali
Author-email: moez.ali@queensu.ca
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: Boruta, cufflinks, gensim, imbalanced-learn, IPython, ipywidgets, joblib, kmodes, lightgbm, matplotlib, mlflow, mlxtend, nltk, numba, pandas, pandas-profiling, plotly, pyLDAvis, pyod, pyyaml, scikit-learn, scikit-plot, scipy, seaborn, spacy, textblob, umap-learn, wordcloud, yellowbrick
Required-by: 


In [ ]:
!pip install numpy==1.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.0-cp38-cp38-manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.0 which is incompatible.
xarray 2022.12.0 requires pandas>=1.3, but you have pandas 1.1.4 which is incompatible.
xarray-einstats 0.4.0 requires scipy>=1.6, but you have scipy 1.5.4 which is incompatible.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.7.1 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.9 which is incompatible.
cmdstanpy 1.0.8 requires numpy>=1.

In [ ]:
import pycaret.classification as py_clf
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_error
from lineartree import LinearTreeRegressor
# import shap
import matplotlib.pyplot as plt

from sdv.tabular import GaussianCopula, TVAE, CTGAN, CopulaGAN

from pandas_profiling import ProfileReport

import numpy as np
import pandas as pd
import seaborn as sns
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Irini/NEW"

Mounted at /content/drive


In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):

  err = (y_true - y_pred)/y_true
  err = err[~np.isinf(err)]  # drop out divison by zero errors 
  abs_err = abs(err)
  mape = 1/len(err) * sum(abs_err)

  return mape

In [ ]:
# load dataset into memory
df = pd.read_csv(f"{path}/NEW_v00_imputed_SMOTE.csv")

df.head()

,NMs,Coating,Crystallinity_%,Aver_crystallite_sizes_nm,Core_size_nm_TEM,Spherical_surface_Area_nm^2 _TEM,Na 1s_Concent_%,O 1s_Concent_%,Ag 3d_Concent_%,C 1s_Concent_%,N 1s_Concent_%,Hydro_size_t0_nm,PI_t0,Hydro_size_t24_nm,PI_t24,Exposure dose_PPM,Organ,Cell_line,multiwell,Assay,viability_%,viability
0,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,254.33,0.38,113.96667,0.5,100.0,Lung,A549,MW6,Alamar_Blue,2.99,very_toxic
1,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,254.33,0.38,113.96667,0.5,100.0,Lung,A549,MW6,Alamar_Blue,3.06,very_toxic
2,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,254.33,0.38,113.96667,0.5,100.0,Lung,A549,MW6,Alamar_Blue,3.09,very_toxic
3,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,173.53,0.42,250.53334,0.4,50.0,Lung,A549,MW6,Alamar_Blue,3.11,very_toxic
4,Ag_HEC_6.4_1.4_sol,HEC,59.9,75.0,14.0,3981.5999,0.0,34.02,0.17,64.51,1.3,173.53,0.42,250.53334,0.4,50.0,Lung,A549,MW6,Alamar_Blue,3.13,very_toxic


In [ ]:
df["N 1s_Concent_%"] = df["N 1s_Concent_%"].astype(float)

In [ ]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NMs                               1682 non-null   object 
 1   Coating                           1682 non-null   object 
 2   Crystallinity_%                   1682 non-null   float64
 3   Aver_crystallite_sizes_nm         1682 non-null   float64
 4   Core_size_nm_TEM                  1682 non-null   float64
 5   Spherical_surface_Area_nm^2 _TEM  1682 non-null   float64
 6   Na 1s_Concent_%                   1682 non-null   float64
 7   O 1s_Concent_%                    1682 non-null   float64
 8   Ag 3d_Concent_%                   1682 non-null   float64
 9   C 1s_Concent_%                    1682 non-null   float64
 10  N 1s_Concent_%                    1682 non-null   float64
 11  Hydro_size_t0_nm                  1682 non-null   float64
 12  PI_t0 

In [ ]:
df.columns

Index(['NMs', 'Coating', 'Crystallinity_%', 'Aver_crystallite_sizes_nm',
       'Core_size_nm_TEM', 'Spherical_surface_Area_nm^2 _TEM',
       'Na 1s_Concent_%', 'O 1s_Concent_%', 'Ag 3d_Concent_%',
       'C 1s_Concent_%', 'N 1s_Concent_%', 'Hydro_size_t0_nm', 'PI_t0',
       'Hydro_size_t24_nm', 'PI_t24', 'Exposure dose_PPM', 'Organ',
       'Cell_line', 'multiwell', 'Assay', 'viability_%', 'viability'],
      dtype='object')

In [ ]:
# set the target variable
target = "viability"

ignore_features = ["NMs",
                   "Organ", 
                   "multiwell",
                   "viability_%"]

# set up dependent variables
categorical_features = ["Coating",
                        "Organ",
                        "Cell_line",
                        "multiwell", 
                        "Assay"]

impute = []

numeric_features = ['Crystallinity_%', 
                    'Aver_crystallite_sizes_nm',
                    'Spherical_surface_Area_nm^2 _TEM',
                    'O 1s_Concent_%', 
                    'Ag 3d_Concent_%',
                    'C 1s_Concent_%', 
                    'N 1s_Concent_%', 
                    'Hydro_size_t0_nm',
                    'PI_t0',
                    'Hydro_size_t24_nm',
                    'PI_t24', 
                    'Exposure dose_PPM']

In [ ]:
preprocess = True

normalize = False
normalize_method = "robust"
remove_outliers = False
remove_perfect_collinearity = False
remove_multicollinearity = False
ignore_low_variance = False

impute = False
impute_method = "iterative"

polynomial_features = False
polynomial_degree = 2

trigonometry_features = False

feature_interaction = False
feature_ratio = False

combine_rare_levels = False

feature_selection = True
feature_selection_method = "boruta"

experiment_setup = py_clf.setup(data=df,
                                target=target,
                                categorical_features=categorical_features,
                                numeric_features=numeric_features,
                                ignore_features=ignore_features,
                                ignore_low_variance=ignore_low_variance,
                                remove_outliers=remove_outliers,
                                remove_multicollinearity=remove_multicollinearity,
                                multicollinearity_threshold=0.9,
                                remove_perfect_collinearity=remove_perfect_collinearity,
                                preprocess=preprocess,
                                normalize=normalize,
                                normalize_method=normalize_method,
                                imputation_type=impute_method,
                                combine_rare_levels=combine_rare_levels,
                                polynomial_features=polynomial_features,
                                polynomial_degree=polynomial_degree,
                                trigonometry_features=trigonometry_features,
                                feature_interaction=feature_interaction,
                                feature_ratio=feature_ratio,
                                feature_selection=feature_selection,
                                feature_selection_method=feature_selection_method,
                                use_gpu=True,
                                silent=True)

,Description,Value
0,session_id,8876
1,Target,viability
2,Target Type,Multiclass
3,Label Encoded,"Toxic: 0, safe: 1, very_toxic: 2"
4,Original Data,"(1682, 22)"
5,Missing Values,False
6,Numeric Features,14
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['Coating',
                                                            'Cell_line',
                                                            'Assay'],
                                      display_types=False,
                                      features_todrop=['NMs', 'Organ',
                                                       'multiwell',
                                                       'viability_%'],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=['Crystallinity_%',
                                                          'Aver_crystallite_sizes_nm',
                                                          'Spherical_su

In [ ]:
py_clf.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8378,0.9446,0.8359,0.8413,0.8373,0.7560,0.7582,0.613
et,Extra Trees Classifier,0.8369,0.9349,0.8351,0.8404,0.8365,0.7548,0.7568,0.607
lightgbm,Light Gradient Boosting Machine,0.8276,0.9486,0.8257,0.8305,0.8268,0.7407,0.7428,0.275
gbc,Gradient Boosting Classifier,0.8191,0.9459,0.8169,0.8215,0.8180,0.7280,0.7300,0.707
dt,Decision Tree Classifier,0.8038,0.8941,0.8014,0.8105,0.8039,0.7049,0.7078,0.021
knn,K Neighbors Classifier,0.7698,0.9081,0.7690,0.7796,0.7708,0.6542,0.6572,0.217
ada,Ada Boost Classifier,0.7307,0.8261,0.7316,0.7407,0.7314,0.5963,0.6000,0.145
lda,Linear Discriminant Analysis,0.6645,0.8447,0.6613,0.6650,0.6597,0.4959,0.4997,0.029
ridge,Ridge Classifier,0.6585,0.0000,0.6526,0.6631,0.6432,0.4852,0.4969,0.018
lr,Logistic Regression,0.6406,0.7918,0.6361,0.6450,0.6319,0.4581,0.4650,0.694


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=8876, verbose=0,
                       warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=8876, verbose=0,
                       warm_start=False)

In [ ]:
model = py_clf.create_model("lightgbm")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8475,0.9423,0.8471,0.8492,0.8480,0.7708,0.7711
1,0.8475,0.9488,0.8454,0.8527,0.8477,0.7703,0.7727
2,0.7627,0.9353,0.7645,0.7637,0.7615,0.6442,0.6457
3,0.8136,0.9446,0.8086,0.8117,0.8081,0.7191,0.7229
4,0.8644,0.9577,0.8625,0.8738,0.8660,0.7957,0.7990
5,0.7627,0.9325,0.7616,0.7697,0.7645,0.6434,0.6447
6,0.8475,0.9525,0.8449,0.8532,0.8479,0.7703,0.7727
7,0.8803,0.9663,0.8774,0.8800,0.8782,0.8201,0.8218
8,0.7863,0.9425,0.7863,0.7845,0.7848,0.6793,0.6798


INFO:logs:create_model_container: 17
INFO:logs:master_model_container: 17
INFO:logs:display_container: 5
INFO:logs:LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=8876, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
INFO:logs:create_model() succesfully completed......................................


In [ ]:
model = py_clf.tune_model(model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7881,0.9275,0.7899,0.7995,0.7890,0.6824,0.6869
1,0.8475,0.9348,0.8471,0.8480,0.8471,0.7707,0.7713
2,0.7966,0.9286,0.7967,0.7980,0.7970,0.6948,0.6951
3,0.7627,0.9368,0.7580,0.7584,0.7590,0.6430,0.6443
4,0.8051,0.9561,0.8040,0.8148,0.8070,0.7068,0.7092
5,0.7881,0.9332,0.7881,0.8017,0.7917,0.6825,0.6853
6,0.8475,0.9463,0.8452,0.8548,0.8470,0.7705,0.7741
7,0.8120,0.9300,0.8094,0.8110,0.8103,0.7174,0.7184
8,0.8034,0.9409,0.8021,0.8026,0.8024,0.7045,0.7051


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 6
INFO:logs:LGBMClassifier(bagging_fraction=0.5, bagging_freq=2, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=21, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=240, n_jobs=-1, num_leaves=40, objective=None,
               random_state=8876, reg_alpha=0.01, reg_lambda=0.0001,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
INFO:logs:tune_model() succesfully completed......................................


In [ ]:
py_clf.evaluate_model(model)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=LGBMClassifier(bagging_fraction=0.5, bagging_freq=2, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=21, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=240, n_jobs=-1, num_leaves=40, objective=None,
               random_state=8876, reg_alpha=0.01, reg_lambda=0.0001,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
model = py_clf.finalize_model(model)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=LGBMClassifier(bagging_fraction=0.5, bagging_freq=2, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=21, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=240, n_jobs=-1, num_leaves=40, objective=None,
               random_state=8876, reg_alpha=0.01, reg_lambda=0.0001,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0), fit_kwargs=None, groups=None, model_only=True, display=None, experiment_custom_tags=None, return_train_score=False)
INFO:logs:Finalizing LGBMClassifier(bagging_fraction=0.5, bagging_freq=2, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.4, max_depth=-1,
      

[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 

INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Finalizing model
INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 5
INFO:logs:LGBMClassifier(bagging_fraction=0.5, bagging_freq=2, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=21, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=240, n_jobs=-1, num_leaves=40, objective=None,
               random_state=8876, reg_alpha=0.01, reg_lambda=0.0001,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)
INFO:logs:create_model() succesfully completed......................................
INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 4
INFO:logs:LGBMClassifier(bagging_fraction

In [ ]:
py_clf.save_model(model, "nano_viability_classifier")

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=LGBMClassifier(bagging_fraction=0.5, bagging_freq=2, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
               importance_type='split', learning_rate=0.4, max_depth=-1,
               min_child_samples=21, min_child_weight=0.001, min_split_gain=0.1,
               n_estimators=240, n_jobs=-1, num_leaves=40, objective=None,
               random_state=8876, reg_alpha=0.01, reg_lambda=0.0001,
               silent='warn', subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0), model_name=nano_viability_classifier, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=['Coating',
                                                            'Cell_line',
                                                            'Assay'],
                                      display_types=False,
          

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=['Coating',
                                                             'Cell_line',
                                                             'Assay'],
                                       display_types=False,
                                       features_todrop=['NMs', 'Organ',
                                                        'multiwell',
                                                        'viability_%'],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=['Crystallinity_%',
                                                           'Aver_crystallite_sizes_nm',
                                                           'Spherical_surface_Area_nm^2 '
                                                           '_TEM',
              